In [1]:
import numpy as np
import pandas as pd

import cv2
import os
import glob
import helper_functions_07b as hf

from multiprocessing import Pool

In [2]:
DATE = ['20230302']#, '20230305']
SESSION = ['SM_Lek1']#, 'SE_Lek1']
DRONE = ['P1D1', 'P1D2', 'P2D3', 'P2D4', 'P3D5', 'P3D6']

files_directory = '/Volumes/EAS_shared/blackbuck/working/processed/Field_Recording_2023/TestRegistration'
video_directory = '/Volumes/EAS_shared/blackbuck/working/rawdata/Field_Recording_2023/Field_Recording_2023/Original/lekking'

In [3]:
# Generate all tasks
tasks = [(date, session, drone, files_directory, video_directory) for date in DATE for session in SESSION for drone in DRONE]

# Use multiprocessing Pool to parallelize
with Pool(processes=6) as pool:  # Adjust the number of processes based on your CPU
    pool.map(hf.process_task, tasks)

20230302 SM_Lek1 P3D5
Total frames in video: 5804
Frame extraction completed for video 20230302_SM_Lek1_P3D5_DJI_0147
Total frames in video: 5815
Frame extraction completed for video 20230302_SM_Lek1_P3D5_DJI_0148
20230302 SM_Lek1 P2D3
Total frames in video: 5804
Frame extraction completed for video 20230302_SM_Lek1_P2D3_DJI_0897
Total frames in video: 5816
Frame extraction completed for video 20230302_SM_Lek1_P2D3_DJI_0898
20230302 SM_Lek1 P1D1
Total frames in video: 5804
Frame extraction completed for video 20230302_SM_Lek1_P1D1_DJI_0869
Total frames in video: 5816
Frame extraction completed for video 20230302_SM_Lek1_P1D1_DJI_0870
20230302 SM_Lek1 P2D4
Total frames in video: 5805
Frame extraction completed for video 20230302_SM_Lek1_P2D4_DJI_0568
Total frames in video: 5815
Frame extraction completed for video 20230302_SM_Lek1_P2D4_DJI_0569
20230302 SM_Lek1 P3D6
Total frames in video: 5802
Frame extraction completed for video 20230302_SM_Lek1_P3D6_DJI_0540
Total frames in video: 581